In [206]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.python.training import training_util

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

1.4.0


In [207]:
df = pd.read_csv("GRIR_GCP_Data.csv", sep=",")

In [208]:
df.head(10)

,WERKS,SCENARIO,KTOKK,VSTATU,VPATD,EKORG,EKGRP,TOTGRQTY,TOTIRQTY,NODLGR,NODLIR,DIFGRIRD,DIFGRIRV,STATUS
0,ML01,3,1,1,30,1,A,0,80,0,90,-80,-38100,1
1,ML01,3,1,1,30,1,A,0,107,0,177,-107,-41600,0
2,ML01,3,1,1,30,1,A,0,107,0,152,-107,-27600,1
3,ML01,3,1,1,30,1,A,0,96,0,79,-96,-13800,1
4,ML01,3,1,1,30,1,A,0,146,0,192,-146,-73500,0
5,ML01,3,1,1,60,1,A,0,189,0,139,-189,-26600,1
6,ML01,3,1,1,60,1,A,0,183,0,48,-183,-69200,0
7,ML01,3,1,1,60,1,A,0,159,0,195,-159,-73600,0
8,ML01,3,1,1,60,1,A,0,185,0,79,-185,-59500,0
9,ML01,3,1,1,60,1,A,0,91,0,168,-91,-4700,1


In [209]:
df.describe()

,SCENARIO,KTOKK,VSTATU,VPATD,EKORG,TOTGRQTY,TOTIRQTY,NODLGR,NODLIR,DIFGRIRD,DIFGRIRV,STATUS
count,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0,8279.0
mean,2.6,1.5,1.1,60.3,1.3,65.9,94.1,103.1,89.5,-28.2,-6716.3,0.4
std,1.1,0.5,0.3,24.4,0.5,63.0,62.4,82.0,75.0,63.2,22797.4,0.5
min,1.0,1.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,-200.0,-75000.0,0.0
25%,2.0,1.0,1.0,30.0,1.0,0.0,54.0,0.0,10.0,-56.0,-9600.0,0.0
50%,3.0,1.0,1.0,60.0,1.0,55.0,99.0,112.0,82.0,-2.0,-546.0,0.0
75%,4.0,2.0,1.0,90.0,2.0,121.0,146.0,172.0,154.0,11.0,5482.0,1.0
max,4.0,2.0,2.0,90.0,2.0,200.0,200.0,268.0,242.0,74.0,59200.0,1.0


In [210]:
df.columns

Index(['WERKS', 'SCENARIO', 'KTOKK', 'VSTATU', 'VPATD', 'EKORG', 'EKGRP',
       'TOTGRQTY', 'TOTIRQTY', 'NODLGR', 'NODLIR', 'DIFGRIRD', 'DIFGRIRV',
       'STATUS'],
      dtype='object')

In [211]:
#Mark some columns as categorical so that TF treats them as categorical
for col_cat in ['SCENARIO','KTOKK','VSTATU','EKORG']:
    df[col_cat] = df[col_cat].astype('str') #Very important to keep this as STR -> Tensorflow treats only STR as categorical
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8279 entries, 0 to 8278
Data columns (total 14 columns):
WERKS       8279 non-null object
SCENARIO    8279 non-null object
KTOKK       8279 non-null object
VSTATU      8279 non-null object
VPATD       8279 non-null int64
EKORG       8279 non-null object
EKGRP       8279 non-null object
TOTGRQTY    8279 non-null int64
TOTIRQTY    8279 non-null int64
NODLGR      8279 non-null int64
NODLIR      8279 non-null int64
DIFGRIRD    8279 non-null int64
DIFGRIRV    8279 non-null int64
STATUS      8279 non-null int64
dtypes: int64(8), object(6)
memory usage: 905.6+ KB


In [212]:
def scale_dataset(df_temp):  
    #Normalize few numerical columns
    for col_norm in ['TOTIRQTY','TOTGRQTY','NODLGR','NODLIR','DIFGRIRD','DIFGRIRV','VPATD']:
        max_value = df_temp[col_norm].max() 
        min_value = df_temp[col_norm].min()
        cols_max[col_norm] = max_value
        cols_min[col_norm] = min_value
        df_temp[col_norm] = (df_temp[col_norm] - min_value) / (max_value - min_value)

    print(cols_max)
    print(cols_min)
    return df_temp

def add_engineered(features):
    #Save value here for 0-1 scaling during training and serving
    cols_max ={'TOTIRQTY':200,'TOTGRQTY':200,'NODLGR':268,'NODLIR':242,'DIFGRIRD':74,'DIFGRIRV':59200,'VPATD':90}
    cols_min ={'TOTIRQTY':0,'TOTGRQTY':0,'NODLGR':0,'NODLIR':0,'DIFGRIRD':-200,'DIFGRIRV':-75000,'VPATD':30}
    
    #Normalize few numerical columns
    for col_norm in ['TOTIRQTY','TOTGRQTY','NODLGR','NODLIR','DIFGRIRD','DIFGRIRV','VPATD']:
        max_value = cols_max[col_norm]
        min_value = cols_min[col_norm]
        features[col_norm] = (features[col_norm] - min_value) / (max_value - min_value)
    
    print(features)
    return features

In [213]:
#Scale dataset
#df_dumm = scale_dataset(df)
#df_dumm.head()
add_engineered(df.iloc[0,:]).head()

c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


WERKS       ML01
SCENARIO       3
KTOKK          1
VSTATU         1
VPATD        0.0
            ... 
NODLGR       0.0
NODLIR       0.4
DIFGRIRD     0.4
DIFGRIRV     0.3
STATUS         1
Name: 0, Length: 14, dtype: object


WERKS       ML01
SCENARIO       3
KTOKK          1
VSTATU         1
VPATD        0.0
Name: 0, dtype: object

In [214]:
def add_features_prepare_train(df_temp):
    #Add any feature engineering or new column here
    return df_temp

In [215]:
#Prepare train and test data set
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]
#evaldf[evaldf['STATUS'] == 1]

In [216]:
# Define your feature columns
def create_feature_cols():
#   lat_buck = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('latitude'), 
#                                                  boundaries = np.arange(32.0, 42, 1).tolist())
#   long_buck = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('longitude'),
#                                                   boundaries = np.arange(1, 52, 1).tolist())
    werks_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='WERKS',
            vocabulary_list=['ML01','ML02','ML03'])
    scenario_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='SCENARIO',
            vocabulary_list=['1','2','3','4'])
    ktokk_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='KTOKK',
            vocabulary_list=['1','2'])    
    vstatu_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='VSTATU',
            vocabulary_list=['1','2'])
    ekorg_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='EKORG',
            vocabulary_list=['1','2'])   
    ekgrp_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='EKGRP',
            vocabulary_list=['A','B','C'])
    return [
        tf.feature_column.indicator_column(werks_c),
        tf.feature_column.indicator_column(scenario_c),
        tf.feature_column.indicator_column(ktokk_c),
        tf.feature_column.indicator_column(vstatu_c),
        tf.feature_column.indicator_column(ekorg_c),
        tf.feature_column.indicator_column(ekgrp_c),
        tf.feature_column.numeric_column('VPATD'),
        tf.feature_column.numeric_column("TOTGRQTY"),
        tf.feature_column.numeric_column("TOTIRQTY"),
        tf.feature_column.numeric_column("NODLGR"),
        tf.feature_column.numeric_column("NODLIR"),
        tf.feature_column.numeric_column("DIFGRIRD"),
        tf.feature_column.numeric_column("DIFGRIRV")
  ]

In [217]:
# Create pandas input function for training
def make_input_fn(df_f, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = add_features_prepare_train(df_f),
    y = df_f['STATUS'],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 200,
    num_threads = 1
  )

#Input Function for Train and Validation Set
def get_validation_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
    x = add_features_prepare_train(df_f),
    y = df_f['STATUS'],
      num_epochs=num_epochs,
      shuffle=shuffle)

# Serving function for external call
def serving_fn():
    feature_placeholders  = {'WERKS' : tf.placeholder(tf.string, [None]),
            'SCENARIO' : tf.placeholder(tf.string, [None]),
            'KTOKK' : tf.placeholder(tf.string, [None]),
            'VSTATU' : tf.placeholder(tf.string, [None]),
            'EKORG' : tf.placeholder(tf.string, [None]),
            'EKGRP' : tf.placeholder(tf.string, [None]),
            'VPATD' : tf.placeholder(tf.float32, [None]),
            'TOTGRQTY' : tf.placeholder(tf.float32, [None]),
            'TOTIRQTY' : tf.placeholder(tf.float32, [None]),
            'NODLGR' : tf.placeholder(tf.float32, [None]),
            'NODLIR' : tf.placeholder(tf.float32, [None]),
            'DIFGRIRD' : tf.placeholder(tf.float32, [None]),
            'DIFGRIRV' : tf.placeholder(tf.float32, [None])
    }

    #Features with transformation logic
    features = {
                key: tf.expand_dims(tensor, -1)
                for key, tensor in feature_placeholders.items()
            }
    
    return tf.estimator.export.ServingInputReceiver(add_engineered(features), feature_placeholders )
serving_fn()

{'NODLIR': <tf.Tensor 'truediv_105:0' shape=(?, 1) dtype=float32>, 'DIFGRIRV': <tf.Tensor 'truediv_107:0' shape=(?, 1) dtype=float32>, 'TOTGRQTY': <tf.Tensor 'truediv_103:0' shape=(?, 1) dtype=float32>, 'DIFGRIRD': <tf.Tensor 'truediv_106:0' shape=(?, 1) dtype=float32>, 'VPATD': <tf.Tensor 'truediv_108:0' shape=(?, 1) dtype=float32>, 'WERKS': <tf.Tensor 'ExpandDims_51:0' shape=(?, 1) dtype=string>, 'SCENARIO': <tf.Tensor 'ExpandDims_52:0' shape=(?, 1) dtype=string>, 'KTOKK': <tf.Tensor 'ExpandDims_53:0' shape=(?, 1) dtype=string>, 'EKORG': <tf.Tensor 'ExpandDims_55:0' shape=(?, 1) dtype=string>, 'TOTIRQTY': <tf.Tensor 'truediv_102:0' shape=(?, 1) dtype=float32>, 'EKGRP': <tf.Tensor 'ExpandDims_58:0' shape=(?, 1) dtype=string>, 'NODLGR': <tf.Tensor 'truediv_104:0' shape=(?, 1) dtype=float32>, 'VSTATU': <tf.Tensor 'ExpandDims_54:0' shape=(?, 1) dtype=string>}


ServingInputReceiver(features={'NODLIR': <tf.Tensor 'truediv_105:0' shape=(?, 1) dtype=float32>, 'DIFGRIRV': <tf.Tensor 'truediv_107:0' shape=(?, 1) dtype=float32>, 'TOTGRQTY': <tf.Tensor 'truediv_103:0' shape=(?, 1) dtype=float32>, 'DIFGRIRD': <tf.Tensor 'truediv_106:0' shape=(?, 1) dtype=float32>, 'VPATD': <tf.Tensor 'truediv_108:0' shape=(?, 1) dtype=float32>, 'WERKS': <tf.Tensor 'ExpandDims_51:0' shape=(?, 1) dtype=string>, 'SCENARIO': <tf.Tensor 'ExpandDims_52:0' shape=(?, 1) dtype=string>, 'KTOKK': <tf.Tensor 'ExpandDims_53:0' shape=(?, 1) dtype=string>, 'EKORG': <tf.Tensor 'ExpandDims_55:0' shape=(?, 1) dtype=string>, 'TOTIRQTY': <tf.Tensor 'truediv_102:0' shape=(?, 1) dtype=float32>, 'EKGRP': <tf.Tensor 'ExpandDims_58:0' shape=(?, 1) dtype=string>, 'NODLGR': <tf.Tensor 'truediv_104:0' shape=(?, 1) dtype=float32>, 'VSTATU': <tf.Tensor 'ExpandDims_54:0' shape=(?, 1) dtype=string>}, receiver_tensors={'NODLIR': <tf.Tensor 'Placeholder_316:0' shape=(?,) dtype=float32>, 'DIFGRIRV': <

In [218]:
#Only DNN Deep wide classifier
def get_deep_wide_features():
    werks_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='WERKS',
            vocabulary_list=['ML01','ML02','ML03'])
    scenario_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='SCENARIO',
            vocabulary_list=['1','2','3','4'])
    ktokk_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='KTOKK',
            vocabulary_list=['1','2'])    
    vstatu_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='VSTATU',
            vocabulary_list=['1','2'])
    ekorg_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='EKORG',
            vocabulary_list=['1','2'])   
    ekgrp_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='EKGRP',
            vocabulary_list=['A','B','C'])
    deep_columns = [
        tf.feature_column.numeric_column('VPATD'),
        tf.feature_column.numeric_column("TOTGRQTY"),
        tf.feature_column.numeric_column("TOTIRQTY"),
        tf.feature_column.numeric_column("NODLGR"),
        tf.feature_column.numeric_column("NODLIR"),
        tf.feature_column.numeric_column("DIFGRIRD"),
        tf.feature_column.numeric_column("DIFGRIRV")
    ]
    wide_columns = [
        tf.feature_column.indicator_column(werks_c),
        tf.feature_column.indicator_column(scenario_c),
        tf.feature_column.indicator_column(ktokk_c),
        tf.feature_column.indicator_column(vstatu_c),
        tf.feature_column.indicator_column(ekorg_c),
        tf.feature_column.indicator_column(ekgrp_c),
    ]
    return deep_columns, wide_columns

def train_and_evaluate_deep(output_dir, num_train_steps):    
##### Create Canned estimator instance
    #Get features
    deep, wide = get_deep_wide_features()

    estimator = tf.estimator.DNNLinearCombinedClassifier(
                                    linear_feature_columns = wide,
                                    dnn_feature_columns = deep,
                                    n_classes=2,
                                    dnn_hidden_units = [32,64,64,64,64,64,
                                                        64,64,64,64,64,64,
                                                        64,64,64,64,64,64,
                                                        32],
                                    dnn_dropout = 0.1,
                                    dnn_optimizer=tf.train.AdamOptimizer(learning_rate=0.0001))
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, None), 
                                      max_steps = num_train_steps)
    exp = tf.estimator.LatestExporter("decision", serving_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(evaldf, 1), 
                                    steps = None, 
                                    exporters = exp,
                                    start_delay_secs = 1, # start evaluating after N seconds, 
                                    throttle_secs = 40)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [219]:
#setup exponential decay function for learning rates
def get_global_step():
    if tf.train.get_global_step() is None:
        return 1
    else:
        return tf.train.get_global_step()

def get_exp_decay_optimizer():
    def exp_decay(global_step=global_step):
        return tf.train.exponential_decay(
          learning_rate=0.0001, global_step=global_step,
          decay_steps=100, decay_rate=0.8, staircase=True)
    # use customized decay function in learning_rate
    return tf.train.AdagradOptimizer(learning_rate=exp_decay(tf.train.get_global_step()))

# setup stepwise decay function for learning rates
def get_stepw_decay_optimizer2():
    def step_decay():
        return tf.train.piecewise_constant(get_global_step(), 
                                           boundaries=[10000,20000,50000,100000], 
                                           values=[0.001,0.0005,0.0001,0.00005,0.00002])
    # use customized decay function in learning_rate
    return tf.train.AdamOptimizer(learning_rate=step_decay)#,epsilon=0.001)

# setup stepwise decay function for learning rates
def get_stepw_decay_optimizer():
    def step_decay(global_step):
        return tf.train.piecewise_constant(global_step, boundaries=[10000,20000,50000,100000], values=[0.001,0.0005,0.0001,0.00005,0.00002])
                                         # use customized decay function in learning_rate
    return tf.train.AdamOptimizer(learning_rate=step_decay(tf.train.get_global_step()))#,epsilon=0.001)

In [220]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):    
##### Create Canned estimator instance
#   estimator = tf.estimator.LinearClassifier(feature_columns = create_feature_cols())
    estimator = tf.estimator.DNNClassifier(feature_columns=create_feature_cols(),
                                          n_classes=2,
                                          hidden_units=[32,64,64,64,64,64,
                                                        64,64,64,64,64,64,
                                                        64,64,64,64,64,64,
                                                        32],
                                          dropout = 0.1,
                                          #optimizer=get_stepw_decay_optimizer)
                                          optimizer=tf.train.AdamOptimizer(learning_rate=0.0003))
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(add_engineered(traindf), None), 
                                      max_steps = num_train_steps)
    exp = tf.estimator.LatestExporter("decision", serving_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(add_engineered(evaldf), 1), 
                                    steps = None, 
                                    exporters = exp,
                                    start_delay_secs = 1, # start evaluating after N seconds, 
                                    throttle_secs = 40)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [221]:
# estimator = tf.estimator.DNNClassifier(feature_columns=create_feature_cols(),
#                                           n_classes=2,
#                                           hidden_units=[32,64,64,64,64,64,
#                                                         64,64,64,64,64,64,
#                                                         64,64,64,64,64,64,
#                                                         32],
#                                           dropout = 0.1,
#                                           #optimizer=get_stepw_decay_optimizer)
#                                           optimizer=tf.train.AdamOptimizer(learning_rate=0.0003))
# estimator.train(make_input_fn(traindf, None), steps=20000)

In [222]:
train_and_evaluate(None, num_train_steps=200000)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': 'C:\\Users\\hrafiq\\AppData\\Local\\Temp\\tmpyp5invlf', '_tf_random_seed': None, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E8D3E4A588>, '_task_id': 0, '_save_summary_steps': 100, '_task_type': 'worker', '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_is_chief': True, '_num_worker_replicas': 1, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_checkpoints_secs': 600}


INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_model_dir': 'C:\\Users\\hrafiq\\AppData\\Local\\Temp\\tmpyp5invlf', '_tf_random_seed': None, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E8D3E4A588>, '_task_id': 0, '_save_summary_steps': 100, '_task_type': 'worker', '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_is_chief': True, '_num_worker_replicas': 1, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_checkpoints_secs': 600}
c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     WERKS SCENARIO KTOKK VSTATU  VPATD EKORG EKGRP  TOTGRQTY  TOTIRQTY  \
0     ML01        3     1      1    0.0     1     A       0.0       0.4   
1     ML01        3     1      1    0.0     1     A       0.0       0.5   
2     ML01        3     1      1    0.0     1     A       0.0       0.5   
3     ML01        3     1      1    0.0     1     A       0.0       0.5   
4     ML01        3     1      1    0.0     1     A       0.0       0.7   
...    ...      ...   ...    ...    ...   ...   ...       ...       ...   
8269  ML03        2     2      1    0.5     2     C       0.8       0.8   
8270  ML03        2     2      1    0.5     2     C       0.5       0.4   
8271  ML03        2     2      1    0.5     2     C       0.7       0.6   
8272  ML03        2     2      1    0.5     2     C       0.5       0.4   
8274  ML03        2     2      1    1.0     2     C       0.8       0.7   

      NODLGR  NODLIR  DIFGRIRD  DIFGRIRV  STATUS  
0        0.0     0.4       0.4       0.3       1

...

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: b"C:\\Users\\hrafiq\\AppData\\Local\\Temp\\tmpyp5invlf\\export\\decision\\temp-b'1531684233'\\saved_model.pb"


INFO:tensorflow:SavedModel written to: b"C:\\Users\\hrafiq\\AppData\\Local\\Temp\\tmpyp5invlf\\export\\decision\\temp-b'1531684233'\\saved_model.pb"


In [ ]:
accuracy_score = classifier.evaluate(input_fn=get_input_fn(training_set, num_epochs=1, shuffle=False))["accuracy"]
print("\nTrain Accuracy: {0:f}\n".format(accuracy_score))

In [ ]:
accuracy_score = classifier.evaluate(input_fn=get_validation_input_fn(test_set, num_epochs=1, shuffle=False))["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

In [ ]:
#Run model on Prediction data set
predictions = classifier.predict_classes(input_fn=get_prediction_input_fn(test_set))
for p in list(predictions):
    print(p)

In [ ]:
for i in list(test_set['STATUS']):
    print(i)